In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv(r'datasets/dataset_compressible_flow_60M_training_nstep180.csv')

# Extract data from 50 million to 55 million (assuming row indices)
test_df = df.iloc[50_000_000:55_000_000]

# Save as test dataset
test_df.to_csv('datasets/dataset_compressible_flow_5M_test.csv', index=False)

In [ ]:
import csv

csv_path = r'datasets\dataset_compressible_flow_60M_training_nstep180.csv'

with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    headers = next(reader)
    print("Headings:", headers)

## Create prediction dataset

In [1]:
import sys
from pathlib import Path

project_root = Path.cwd().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [2]:
import pandas as pd
import torch
from src.evaluation import load_model
from config import load_config
from utils.metrics import compute_regression_metrics

shear_exp_dir = project_root / r"C:\Users\cervinka\cervinka\GitHub\MathCAS\outputs\2025-08-26_12-47-54_shear_dan_750"
shear_config = load_config(shear_exp_dir / "config.yaml")
shear_model = load_model(shear_config, shear_exp_dir)
shear_model.eval()

res_exp_dir = project_root / r"C:\Users\cervinka\cervinka\GitHub\MathCAS\outputs\2025-08-27_11-17-55_res_trueShear_dan_750"
res_config = load_config(res_exp_dir / "config.yaml")
res_model = load_model(res_config, res_exp_dir)
res_model.eval()

chunk_size = 100_000  # Adjust as needed for your memory
input_path = r"C:\Users\cervinka\cervinka\GitHub\MathCAS\datasets\dataset_compressible_flow_eda_50M_training_nstep180.csv"
output_path = r"C:\Users\cervinka\cervinka\GitHub\MathCAS\datasets\dataset_compressible_flow_eda_50M_training_nstep180_with_predshear.csv"

first = True
for chunk in pd.read_csv(input_path, chunksize=chunk_size):
    # Predict shear
    shear_inputs = chunk[shear_config.data.in_cols].values
    shear_tensor = torch.tensor(shear_inputs, dtype=torch.float32)
    with torch.no_grad():
        pred_shear = shear_model(shear_tensor).squeeze().numpy()
    chunk["PredShear"] = pred_shear

    # Predict with second model
    res_inputs = chunk[res_config.data.in_cols].values
    res_tensor = torch.tensor(res_inputs, dtype=torch.float32)
    with torch.no_grad():
        res_pred = res_model(res_tensor).squeeze().numpy()

    # Print metrics for this chunk
    output_cols = res_config.data.out_cols
    y_true = chunk[output_cols].values
    y_pred = res_pred if res_pred.ndim == 2 else res_pred.reshape(-1, 1)
    for i, col in enumerate(output_cols):
        y_true_col = y_true[:, i]
        y_pred_col = y_pred[:, i]
        metrics = compute_regression_metrics(y_true_col, y_pred_col)
        print(f"\nChunk metrics for output '{col}':")
        for k, v in metrics.items():
            print(f"  {k}: {v}")

    # Save chunk with PredShear
    chunk.to_csv(output_path, mode='w' if first else 'a', header=first, index=False)
    first = False


Chunk metrics for output 'ResVort':
  MAPE: 5.7921e+01
  MAE: 1.8274e-03
  MSE: 1.3476e-05
  MedAE: 1.9245e-04
  MaxAE: 1.9081e-01

Chunk metrics for output 'ResStrain':
  MAPE: 9.0145e+00
  MAE: 3.1179e-03
  MSE: 2.2684e-05
  MedAE: 2.3854e-03
  MaxAE: 3.6900e-01

Chunk metrics for output 'ResVort':
  MAPE: 2.1009e+01
  MAE: 1.8359e-03
  MSE: 1.3066e-05
  MedAE: 1.8863e-04
  MaxAE: 1.5711e-01

Chunk metrics for output 'ResStrain':
  MAPE: 9.4197e+00
  MAE: 3.1215e-03
  MSE: 2.1647e-05
  MedAE: 2.3916e-03
  MaxAE: 2.8033e-01

Chunk metrics for output 'ResVort':
  MAPE: 1.0284e+02
  MAE: 1.8388e-03
  MSE: 1.2913e-05
  MedAE: 1.9546e-04
  MaxAE: 1.4725e-01

Chunk metrics for output 'ResStrain':
  MAPE: 9.2201e+00
  MAE: 3.1122e-03
  MSE: 2.0362e-05
  MedAE: 2.3894e-03
  MaxAE: 2.7756e-01

Chunk metrics for output 'ResVort':
  MAPE: 2.4014e+01
  MAE: 1.8375e-03
  MSE: 1.3114e-05
  MedAE: 1.8299e-04
  MaxAE: 1.4659e-01

Chunk metrics for output 'ResStrain':
  MAPE: 9.5408e+00
  MAE: 3.131